In [72]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
import os
import Directory
import transformation
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [73]:
# Ignore warnings

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [157]:
###### Imports ######

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [158]:
import importlib
importlib.reload(transformation)

<module 'transformation' from 'C:\\Users\\tanse\\Documents\\MyStuff\\Temp\\Kaggle\\titanic\\Code\\transformation.py'>

In [159]:
variables = 'Sex_pclass_cabinexist_isoap_ischild'

### Logistic Regression

In [160]:
###### Name of output File ######
file_name = 'LogisticRegression_' + variables + '.csv'

In [161]:
###### Get the data and transform #######

titanicTrain = pd.read_csv(os.path.join(Directory.dataPath,'train.csv'))
X_test = pd.read_csv(os.path.join(Directory.dataPath,'test.csv'))

titanicCleaned = titanicTrain.copy()
titanicCleaned = transformation.fillTitanicNa(titanicCleaned,titanicCleaned,True,True)

# This is the features data from the training set without the PassengerId
X = titanicCleaned.drop(['Survived','PassengerId'],axis=1)

# This is the response variable from the training set
y = titanicCleaned['Survived']

# This is the test data including the PassengerId
test = pd.read_csv(os.path.join(Directory.dataPath,'test.csv'))

# This is the test data without the PassengerId
test2 = test.drop(['PassengerId'],axis=1)

In [162]:
###### Show the columns ######
X.columns

Index(['CabinExistence', 'IsOAP', 'IsChild', 'male', 'pclass__2', 'pclass__3'], dtype='object')

In [163]:
X

,CabinExistence,IsOAP,IsChild,male,pclass__2,pclass__3
0,False,0,0,1,0,1
1,True,0,0,0,0,0
2,False,0,0,0,0,1
3,True,0,0,0,0,0
4,False,0,0,1,0,1
...,...,...,...,...,...,...
886,False,0,0,1,1,0
887,True,0,0,0,0,0
888,False,0,0,0,0,1
889,True,0,0,1,0,0


In [164]:
###### Get best parameters ######

# The parameters of the Logistic Regression model which we want to iterate over
param_grid = {'penalty':['l2','l1',None], 'solver':['liblinear','newton-cg', 'lbfgs', 'sag', 'saga'],\
              'C':[10000000,1000,500,200,100,20,10,2,1,0.1,0.01,0.001],\
              'fit_intercept':[True,False]}

# Utilise GridSearchCV to run Linear Regression where for each set of parameters it will run cross validation 20 times.
# n_jobs is the number of parallel jobs
lg_search = GridSearchCV(LogisticRegression(), param_grid, cv=10, refit=True, verbose=1,n_jobs=4)
lg_search.fit(X,y)
lg_search.best_estimator_

Fitting 10 folds for each of 360 candidates, totalling 3600 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  68 tasks      | elapsed:    2.5s
[Parallel(n_jobs=4)]: Done 3600 out of 3600 | elapsed:    9.6s finished


LogisticRegression(C=0.1, solver='liblinear')

In [165]:
###### Compared optimised model with default model ######

# Default classifier
lg1 = LogisticRegression()

# Optimised classifier
lg2 = lg_search.best_estimator_

# Here we run cross validation for the default model and the optimised model. We run it 200 times
# with a randomised train and test set and get the mean accuracy score
lg1_scores = cross_val_score(lg1, X, y, cv=10, scoring='accuracy',n_jobs=4)
lg2_scores = cross_val_score(lg2, X, y, cv=10, scoring='accuracy',n_jobs=4)
print('The default classifier accuracy score is {}'.format(lg1_scores.mean()))
print('The optimised classifier accuracy score is {}'.format(lg2_scores.mean()))

The default classifier accuracy score is 0.7923470661672909
The optimised classifier accuracy score is 0.7957802746566791


In [166]:
###### Get and Save Predictions ######

# This is the features data from the training set without the PassengerId
X = titanicCleaned.drop(['Survived','PassengerId'],axis=1)

# This is the response variable from the training set
y = titanicCleaned['Survived']

# This is the test data including the PassengerId
test = pd.read_csv(os.path.join(Directory.dataPath,'test.csv'))

# This is the test data without the PassengerId
test2 = test.drop(['PassengerId'],axis=1)

lg2.fit(X,y)

titanicTestCleaned = transformation.fillTitanicNa(test,titanicTrain.copy(),True,True)
titanicTestCleaned.drop('PassengerId',inplace=True,axis=1)
transformation.makeColumnsEqual(titanicTestCleaned,X)

# lg2 was already trained, we use it here in the function to do the prediction
transformation.PredictAndSave2(test2,titanicTrain.copy(),X,lg2,os.path.join(Directory.outputPath,file_name),test).head()

,Survived,CabinExistence,IsOAP,IsChild,male,pclass__2,pclass__3
PassengerId,,,,,,,
892,0,False,0,0,1,0,1
893,0,False,0,0,0,0,1
894,0,False,1,0,1,1,0
895,0,False,0,0,1,0,1
896,0,False,0,0,0,0,1


### Linear SVC

In [167]:
###### Name of output File ######
file_name = 'LSVC_' + variables + '.csv'

In [168]:
###### Get the data and transform #######

titanicTrain = pd.read_csv(os.path.join(Directory.dataPath,'train.csv'))
X_test = pd.read_csv(os.path.join(Directory.dataPath,'test.csv'))

titanicCleaned = titanicTrain.copy()
titanicCleaned = transformation.fillTitanicNa(titanicCleaned,titanicCleaned,True,True)

# This is the features data from the training set without the PassengerId
X = titanicCleaned.drop(['Survived','PassengerId'],axis=1)

# This is the response variable from the training set
y = titanicCleaned['Survived']

# This is the test data including the PassengerId
test = pd.read_csv(os.path.join(Directory.dataPath,'test.csv'))

# This is the test data without the PassengerId
test2 = test.drop(['PassengerId'],axis=1)

In [169]:
###### Show the columns ######
X.columns

Index(['CabinExistence', 'IsOAP', 'IsChild', 'male', 'pclass__2', 'pclass__3'], dtype='object')

In [170]:
###### Get best parameters ######

# The parameters of the Logistic Regression model which we want to iterate over
param_grid = {'dual':[True,False], 'C':[0.00001,0.001,0.01,0.1,0.15,1,2,10,20],'penalty':['l2','l1',None]}

# Utilise GridSearchCV to run Linear Regression where for each set of parameters it will run cross validation 20 times.
# n_jobs is the number of parallel jobs
lsvc_search = GridSearchCV(LinearSVC(), param_grid, cv=10, refit=True, verbose=1,n_jobs=1)
lsvc_search.fit(X,y)
print(lsvc_search.best_estimator_)

Fitting 10 folds for each of 54 candidates, totalling 540 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


LinearSVC(C=0.1, dual=False, penalty='l1')


[Parallel(n_jobs=1)]: Done 540 out of 540 | elapsed:    2.6s finished


In [171]:
###### Compared optimised model with default model ######

# Default classifier
lg1 = LinearSVC()

# Optimised classifier
lg2 = lsvc_search.best_estimator_

# Here we run cross validation for the default model and the optimised model. We run it 200 times
# with a randomised train and test set and get the mean accuracy score
lg1_scores = cross_val_score(lg1, X, y, cv=10, scoring='accuracy',n_jobs=4)
lg2_scores = cross_val_score(lg2, X, y, cv=10, scoring='accuracy',n_jobs=4)
print('The default classifier accuracy score is {}'.format(lg1_scores.mean()))
print('The optimised classifier accuracy score is {}'.format(lg2_scores.mean()))

The default classifier accuracy score is 0.7923470661672909
The optimised classifier accuracy score is 0.7934706616729089


In [172]:
###### Get and Save Predictions ######

# This is the features data from the training set without the PassengerId
X = titanicCleaned.drop(['Survived','PassengerId'],axis=1)

# This is the response variable from the training set
y = titanicCleaned['Survived']

# This is the test data including the PassengerId
test = pd.read_csv(os.path.join(Directory.dataPath,'test.csv'))

# This is the test data without the PassengerId
test2 = test.drop(['PassengerId'],axis=1)

lg2.fit(X,y)

titanicTestCleaned = transformation.fillTitanicNa(test,titanicTrain.copy(),True,True)
titanicTestCleaned.drop('PassengerId',inplace=True,axis=1)
transformation.makeColumnsEqual(titanicTestCleaned,X)

# lg2 was already trained, we use it here in the function to do the prediction
transformation.PredictAndSave2(test2,titanicTrain.copy(),X,lg2,os.path.join(Directory.outputPath,file_name),test).head()

,Survived,CabinExistence,IsOAP,IsChild,male,pclass__2,pclass__3
PassengerId,,,,,,,
892,0,False,0,0,1,0,1
893,1,False,0,0,0,0,1
894,0,False,1,0,1,1,0
895,0,False,0,0,1,0,1
896,1,False,0,0,0,0,1


### Random Forest

In [173]:
###### Name of output File ######
file_name = 'RF_' + variables + '.csv'

In [174]:
###### Get the data and transform #######

titanicTrain = pd.read_csv(os.path.join(Directory.dataPath,'train.csv'))
X_test = pd.read_csv(os.path.join(Directory.dataPath,'test.csv'))

titanicCleaned = titanicTrain.copy()
titanicCleaned = transformation.fillTitanicNa(titanicCleaned,titanicCleaned,True,True)

# This is the features data from the training set without the PassengerId
X = titanicCleaned.drop(['Survived','PassengerId'],axis=1)

# This is the response variable from the training set
y = titanicCleaned['Survived']

# This is the test data including the PassengerId
test = pd.read_csv(os.path.join(Directory.dataPath,'test.csv'))

# This is the test data without the PassengerId
test2 = test.drop(['PassengerId'],axis=1)

In [175]:
###### Show the columns ######
X.columns

Index(['CabinExistence', 'IsOAP', 'IsChild', 'male', 'pclass__2', 'pclass__3'], dtype='object')

In [178]:
###### Get best parameters ######

# The parameters of the Logistic Regression model which we want to iterate over
param_grid = {'n_estimators':range(10,100,10),'max_depth':range(2,10,2)}

# Utilise GridSearchCV to run Linear Regression where for each set of parameters it will run cross validation 20 times.
# n_jobs is the number of parallel jobs
rf_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=10, refit=True, verbose=1,n_jobs=2)
rf_search.fit(X,y)
print(rf_search.best_estimator_)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RandomForestClassifier(max_depth=2, n_estimators=20)


[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed:   26.8s finished


In [179]:
###### Compared optimised model with default model ######

# Default classifier
lg1 = RandomForestClassifier()

# Optimised classifier
lg2 = rf_search.best_estimator_

# Here we run cross validation for the default model and the optimised model. We run it 200 times
# with a randomised train and test set and get the mean accuracy score
lg1_scores = cross_val_score(lg1, X, y, cv=10, scoring='accuracy',n_jobs=4)
lg2_scores = cross_val_score(lg2, X, y, cv=10, scoring='accuracy',n_jobs=4)
print('The default classifier accuracy score is {}'.format(lg1_scores.mean()))
print('The optimised classifier accuracy score is {}'.format(lg2_scores.mean()))

The default classifier accuracy score is 0.7924094881398251
The optimised classifier accuracy score is 0.7991510611735331


In [180]:
###### Get and Save Predictions ######

# This is the features data from the training set without the PassengerId
X = titanicCleaned.drop(['Survived','PassengerId'],axis=1)

# This is the response variable from the training set
y = titanicCleaned['Survived']

# This is the test data including the PassengerId
test = pd.read_csv(os.path.join(Directory.dataPath,'test.csv'))

# This is the test data without the PassengerId
test2 = test.drop(['PassengerId'],axis=1)

lg2.fit(X,y)

titanicTestCleaned = transformation.fillTitanicNa(test,titanicTrain.copy(),True,True)
titanicTestCleaned.drop('PassengerId',inplace=True,axis=1)
transformation.makeColumnsEqual(titanicTestCleaned,X)

# lg2 was already trained, we use it here in the function to do the prediction
transformation.PredictAndSave2(test2,titanicTrain.copy(),X,lg2,os.path.join(Directory.outputPath,file_name),test).head()

,Survived,CabinExistence,IsOAP,IsChild,male,pclass__2,pclass__3
PassengerId,,,,,,,
892,0,False,0,0,1,0,1
893,0,False,0,0,0,0,1
894,0,False,1,0,1,1,0
895,0,False,0,0,1,0,1
896,0,False,0,0,0,0,1


### Extra Trees

In [181]:
###### Name of output File ######
file_name = 'ET_' + variables + '.csv'

In [182]:
###### Get the data and transform #######

titanicTrain = pd.read_csv(os.path.join(Directory.dataPath,'train.csv'))
X_test = pd.read_csv(os.path.join(Directory.dataPath,'test.csv'))

titanicCleaned = titanicTrain.copy()
titanicCleaned = transformation.fillTitanicNa(titanicCleaned,titanicCleaned,True,True)

# This is the features data from the training set without the PassengerId
X = titanicCleaned.drop(['Survived','PassengerId'],axis=1)

# This is the response variable from the training set
y = titanicCleaned['Survived']

# This is the test data including the PassengerId
test = pd.read_csv(os.path.join(Directory.dataPath,'test.csv'))

# This is the test data without the PassengerId
test2 = test.drop(['PassengerId'],axis=1)

In [183]:
###### Show the columns ######
X.columns

Index(['CabinExistence', 'IsOAP', 'IsChild', 'male', 'pclass__2', 'pclass__3'], dtype='object')

In [184]:
###### Get best parameters ######

# The parameters of the Logistic Regression model which we want to iterate over
param_grid = {'n_estimators':range(1,40,4), 'criterion':['gini','entropy'],'max_depth':range(2,10,2)}

# Utilise GridSearchCV to run Linear Regression where for each set of parameters it will run cross validation 20 times.
# n_jobs is the number of parallel jobs
et_search = GridSearchCV(ExtraTreesClassifier(), param_grid, cv=10, refit=True, verbose=1,n_jobs=2)
et_search.fit(X,y)
print(et_search.best_estimator_)

Fitting 10 folds for each of 80 candidates, totalling 800 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done 202 tasks      | elapsed:    4.8s


ExtraTreesClassifier(criterion='entropy', max_depth=2, n_estimators=33)


[Parallel(n_jobs=2)]: Done 800 out of 800 | elapsed:   15.1s finished


In [185]:
###### Compared optimised model with default model ######

# Default classifier
lg1 = ExtraTreesClassifier()

# Optimised classifier
lg2 = et_search.best_estimator_

# Here we run cross validation for the default model and the optimised model. We run it 200 times
# with a randomised train and test set and get the mean accuracy score
lg1_scores = cross_val_score(lg1, X, y, cv=10, scoring='accuracy',n_jobs=4)
lg2_scores = cross_val_score(lg2, X, y, cv=10, scoring='accuracy',n_jobs=4)
print('The default classifier accuracy score is {}'.format(lg1_scores.mean()))
print('The optimised classifier accuracy score is {}'.format(lg2_scores.mean()))

The default classifier accuracy score is 0.7901622971285892
The optimised classifier accuracy score is 0.7935330836454432


In [186]:
###### Get and Save Predictions ######

# This is the features data from the training set without the PassengerId
X = titanicCleaned.drop(['Survived','PassengerId'],axis=1)

# This is the response variable from the training set
y = titanicCleaned['Survived']

# This is the test data including the PassengerId
test = pd.read_csv(os.path.join(Directory.dataPath,'test.csv'))

# This is the test data without the PassengerId
test2 = test.drop(['PassengerId'],axis=1)

lg2.fit(X,y)

titanicTestCleaned = transformation.fillTitanicNa(test,titanicTrain.copy(),True,True)
titanicTestCleaned.drop('PassengerId',inplace=True,axis=1)
transformation.makeColumnsEqual(titanicTestCleaned,X)

# lg2 was already trained, we use it here in the function to do the prediction
transformation.PredictAndSave2(test2,titanicTrain.copy(),X,lg2,os.path.join(Directory.outputPath,file_name),test).head()

,Survived,CabinExistence,IsOAP,IsChild,male,pclass__2,pclass__3
PassengerId,,,,,,,
892,0,False,0,0,1,0,1
893,0,False,0,0,0,0,1
894,0,False,1,0,1,1,0
895,0,False,0,0,1,0,1
896,0,False,0,0,0,0,1
